In [214]:
import pandas as pd
import numpy as np
import ast
import pickle



In [215]:
import platform
print(pd.__version__)
print(platform.python_version())



2.1.4
3.11.7


In [216]:
mv=pd.read_csv("movies.csv")
cd=pd.read_csv("credits.csv")

df=mv.merge(cd,on='title')
# df.head(5)
# mv.head()


In [217]:
df.drop([ 'id', 'spoken_languages','homepage', 'status', 'tagline', 'original_title', 'vote_count'],axis=1,inplace=True)

In [218]:
df.isnull().sum()

budget                  0
genres                  0
keywords                0
original_language       0
overview                3
popularity              0
production_companies    0
production_countries    0
release_date            1
revenue                 0
runtime                 2
title                   0
vote_average            0
movie_id                0
cast                    0
crew                    0
dtype: int64

In [219]:
df.dropna(inplace=True)

In [220]:
df.duplicated().sum()

0

In [221]:
med=df.budget.median()
df['budget']=df['budget'].replace(0,med)


In [222]:
df.insert(0,'movie_id',df.pop('movie_id'))
# df.head(3)

In [223]:
def budget(l1):
    mov_list=[]
    for i in l1:
        mov_dict={
            'name': i,
            'index':df.loc[df['title'] == i].index[0],
            'budget':(list(df.loc[df['title'] == i].budget))
        }
        
        mov_list.append(mov_dict)
        
    mov_list.sort(key=lambda x:x['budget'],reverse=True)
    l2=[]
    for i in mov_list:
        l2.append(i['name'])
    return l2
    

# mov_list=[]
# for i in l1:
#     mov_list.append(list(df.loc[df['title'] == i].index))
#     mov_list.sort(reverse=True)
    
# # print(mov_list)
# for i in range(len(mov_list)):
#     print(df.iloc[i]['title'])
    


In [224]:
def converter(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
def converter_prod_comp(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
        return l
def converter_cast(obj):
    l=[]
    cnt=0
    for i in ast.literal_eval(obj):
        if cnt !=3:
            l.append(i['name'])
            cnt+=1
        else:
            break
    return l
def converter_crew(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director' :
            l.append(i['name'])
            break
    return l
    
# x=ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')
# x[0]['name']

In [225]:
df['cast']=df['cast'].apply(converter_cast)


In [226]:
df['crew']=df['crew'].apply(converter_crew)
# df=df.explode('crew')



In [227]:
df['genres']=df['genres'].apply(converter)

In [228]:
df['keywords']= df['keywords'].apply(converter)

In [229]:
df['production_companies']= df['production_companies'].apply(converter)

In [230]:
df['production_countries']= df['production_countries'].apply(converter_prod_comp)
    

In [231]:
df['popularity']=df['popularity'].astype('int')

In [232]:
df['production_countries'].fillna('NAN',inplace=True)
type(df['production_countries'].values[0])
df['production_countries'] = df['production_countries'].apply(lambda x: x[0] if x else None)

# exploded_list = list(df['production_countries'].values)
# exploded_list

In [233]:
df['genres']=df['genres'].apply(lambda x:[i.replace(' ','')for i in x])
df['keywords']=df['keywords'].apply(lambda x:[i.replace(' ','')for i in x])
df['production_companies']=df['production_companies'].apply(lambda x:[i.replace(' ','_')for i in x])
df['overview']=df['overview'].apply(lambda x:x.split())


In [234]:
def get_unique_elements(df):
    unique_element=set()
    unique_elements = [item for sublist in df for item in (sublist if isinstance(sublist, list) else [sublist])]
    return set(unique_elements)
    
# uni_genre=get_unique_elements(df.genres)
# uni_coun=get_unique_elements(df.production_countries)
# uni_com=get_unique_elements(df.production_companies)


# print(type(uni_coun))

In [235]:
df.original_language.replace({'en': 'English',
        'ja': 'Japanese',
        'fr': 'French',
        'zh': 'Chinese',
        'es': 'Spanish',
        'ko': 'Korean',
        'de': 'German',
        'hi': 'Hindi',
        'ru': 'Russian',
        'te': 'Telugu',
        'cn': 'Cantonese',
        'it': 'Italian',
        'nl': 'Dutch',
        'ta': 'Tamil',
        'sv': 'Swedish',
        'th': 'Thai',
        'da': 'Danish',
        'xx': 'Unknown',
        'hu': 'Hungarian',
        'cs': 'Czech',
        'pt': 'Portuguese',
        'is': 'Icelandic',
        'tr': 'Turkish',
        'nb': 'Norwegian Bokmål',
        'af': 'Afrikaans',
        'pl': 'Polish',
        'he': 'Hebrew',
        'ar': 'Arabic',
        'vi': 'Vietnamese',
        'ky': 'Kyrgyz',
        'id': 'Indonesian',
        'ro': 'Romanian',
        'fa': 'Persian',
        'no': 'Norwegian',
        'sl': 'Slovenian',
        'ps': 'Pashto',
        'el': 'Greek'},inplace=True)

In [236]:
df['tag']=df['overview']+df['genres'] + df['keywords'] + df['cast'] + df['crew']
df['tag']=df['tag'].apply(lambda x:" ".join(x))
df['tag']=df['tag'].apply(lambda x:x.lower())


In [237]:
# Assuming you have a DataFrame 'df' with a column 'column_to_scale'
df['popularity']  =(df['popularity'] - df['popularity'].min()) / (df['popularity'].max() - df['popularity'].min()) * 100



In [238]:
df['popularity']=df['popularity'].astype(int)

In [239]:
df['Profit']=df['revenue']-df['budget']
# df['Profit']
def million(x):
    return x/1000000
df['Profit']=df['Profit'].apply(million)
df['Profit']=df['Profit'].astype(int)


In [240]:
df['budget']=df['budget'].apply(million)
df['budget']=df['budget'].astype(int)

In [241]:
# print(df['release_date'].isnull().sum())
df['release_date'] = pd.to_datetime(df['release_date'])
df['year']=df['release_date'].dt.year
# df['year'].unique()

In [242]:
df['overview']=df['overview'].apply(lambda x:" ".join(x))
# df['overview']

In [243]:
df['crew'] = df['crew'].apply(lambda x: x[0] if x else None)
# df['crew']

In [244]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(df['tag']).toarray()
cv.get_feature_names_out(input_features=None)

array(['000', '007', '10', ..., 'zooey', 'zucker', 'zwick'], dtype=object)

In [245]:
cv.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [246]:
import nltk
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()


In [247]:
def stem(text):
    y=[]

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)
df['tag']=df['tag'].apply(stem)
# stem(df['tag'][0])


In [248]:
from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(vectors)
# print(similarity[0])
# sorted(enumerate(similarity[0]),reverse=True,key=lambda x:x[1])[1:6]

def recommend(movie):
    movie_index = df[df['title']== movie].index[0]
    # print(movie_index)
    distances=similarity[movie_index]
    movie_list=sorted(enumerate(distances),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(df.iloc[i[0]].title)
        # print(i)
    return


In [249]:
# df.head(5).tag[0]
recommend('Avatar')

# df.columns
# df.overview

Aliens
Independence Day
Titan A.E.
Aliens vs Predator: Requiem
Battle: Los Angeles


In [250]:
df.to_csv("Final.csv")


In [251]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [252]:
pickle.dump(df,open('movies.pkl','wb'))



In [253]:

def filter_movies_by_type(df, input_list,column):
    return df[df[column].apply(lambda x: all(element in x for element in input_list))]

input_genres = ["Comedy",'Romance','Drama','Thriller']


filtered_movies = filter_movies_by_type(df, input_genres,'genres')
filtered_movies.original_language

1205    English
1694    English
2054    English
2134    English
Name: original_language, dtype: object

In [254]:

# start_year=2016
# end_year=2017
# filtered_df = df[df['year'].between(start_year, end_year)]
# for i in filtered_df.movie_id:
#     print(df[df['movie_id']==i].title.values[0])


In [255]:
# df.loc[df['original_language'] == 'English' ]['title']
# 

In [256]:
genre = ["Comedy",'Romance','Drama','Thriller']
# genre = ['Drama']

Lang = ['English']

filtered_df = df[df['year'].between(2000, 2010)]
filtered_genre= filter_movies_by_type(filtered_df, genre,'genres')
filtered_lang= filter_movies_by_type(filtered_genre, Lang,'original_language')
filtered_lang



,movie_id,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,cast,crew,tag,Profit,year
1694,4912,30,"[Comedy, Crime, Drama, Romance, Thriller]","[microfilm, biography, silencer, intrigue]",English,"Television made him famous, but his biggest hi...",2,"[Miramax_Films, Allied_Filmmakers, Mad_Chance]",United States of America,2002-12-30,33013805,113.0,Confessions of a Dangerous Mind,6.6,"[Sam Rockwell, Drew Barrymore, Julia Roberts]",George Clooney,"televis made him famous, but hi biggest hit ha...",3,2002
2134,9895,20,"[Comedy, Drama, Romance, Thriller]","[usapresident, presidentialelection, comedian]",English,The irreverent host of a political satire talk...,1,"[Universal_Pictures, Morgan_Creek_Productions]",United States of America,2006-10-09,41237648,115.0,Man of the Year,5.8,"[Robin Williams, Christopher Walken, Laura Lin...",Barry Levinson,the irrever host of a polit satir talk show de...,21,2006


In [257]:
countries_list=['United States of America']

filtered_coun = filtered_lang[filtered_lang['production_countries'].isin(countries_list)]
x=filtered_coun.title

(list_of_movies) = x.values.tolist()
print(len(list_of_movies))
# 

2


In [258]:
filtered_coun = filtered_lang[filtered_lang['production_countries'].apply(lambda x: any(country in x for country in countries))]

In [263]:
df.loc[df['crew']=="Christopher Nolan"].count()

movie_id                0
budget                  0
genres                  0
keywords                0
original_language       0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
title                   0
vote_average            0
cast                    0
crew                    0
tag                     0
Profit                  0
year                    0
dtype: int64